# Dataset Management: Split & Merge

Hier können wir den Datensatz aufteilen und wieder zusammenfügen.

**Funktionen:**
* **SPLIT:** Nimmt die Bilder aus `data/[Klasse]` und verteilt sie zufällig auf:
  * `data/train` (70%)
  * `data/val` (15%)
  * `data/test` (15%)
* **MERGE:** Macht das rückgängig. Schiebt alles zurück in `data/[Klasse]`, damit du z.B. neue Bilder hinzufügen und neu splitten kannst.

**Wichtig:** Der Ordner `data/all` wird dabei komplett ignoriert und bleibt unberührt.

In [1]:
import os
import shutil
import random
from pathlib import Path

# --- KONFIGURATION ---
DATA_DIR = Path("dataset/data")
IGNORED_FOLDERS = ["all", "train", "val", "test", ".ipynb_checkpoints"]

def split_data(train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """Verteilt Daten in train/val/test Unterordner."""

    # Sicherheitscheck: Gibt es schon einen Split?
    if (DATA_DIR / "train").exists():
        print("⚠️ Es sieht so aus, als wären die Daten schon gesplittet!")
        print("Bitte führe erst 'merge_data()' aus, bevor du neu splittest.")
        return

    # 1. Klassen finden (ignoriere 'all', 'train' etc.)
    classes = [d for d in DATA_DIR.iterdir() if d.is_dir() and d.name not in IGNORED_FOLDERS]

    if not classes:
        print("❌ Keine Klassen-Ordner in 'data/' gefunden.")
        return

    print(f"Gefundene Klassen zum Splitten: {[c.name for c in classes]}")
    print(f"Split-Verhältnis: {train_ratio*100}% / {val_ratio*100}% / {test_ratio*100}%\n")

    for class_dir in classes:
        class_name = class_dir.name

        # Alle Bilder holen
        images = [f for f in class_dir.glob("*") if f.is_file()]
        random.shuffle(images) # Zufällig mischen

        # Anzahl berechnen
        n = len(images)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)
        # Der Rest geht ins Test-Set (um Rundungsfehler zu vermeiden)

        # Slices definieren
        train_imgs = images[:n_train]
        val_imgs = images[n_train:n_train + n_val]
        test_imgs = images[n_train + n_val:]

        # Verschieben
        # Helper Funktion zum Verschieben
        def move_files(file_list, target_split):
            target_dir = DATA_DIR / target_split / class_name
            target_dir.mkdir(parents=True, exist_ok=True)
            for img in file_list:
                shutil.move(str(img), str(target_dir / img.name))

        move_files(train_imgs, "train")
        move_files(val_imgs, "val")
        move_files(test_imgs, "test")

        print(f"  -> {class_name}: {len(train_imgs)} Train, {len(val_imgs)} Val, {len(test_imgs)} Test")

        # Leeren Klassen-Ordner löschen (optional, hält es sauber)
        # os.rmdir(class_dir)

    print("\n✅ SPLIT ABGESCHLOSSEN. Ordnerstruktur ist jetzt bereit für Training.")


def merge_data():
    """Holt alle Daten aus train/val/test zurück in den Hauptordner."""

    splits = ["train", "val", "test"]

    # Prüfen ob überhaupt was zum Mergen da ist
    if not any((DATA_DIR / s).exists() for s in splits):
        print("⚠️ Nichts zu mergen (keine train/val/test Ordner gefunden).")
        return

    print("Starte Merge (Rückgängig machen)...")

    for split in splits:
        split_path = DATA_DIR / split
        if split_path.exists():
            # Gehe durch die Klassen im Split-Ordner (z.B. data/train/cup)
            for class_dir in split_path.iterdir():
                if class_dir.is_dir():
                    class_name = class_dir.name

                    # Ziel: data/cup
                    target_dir = DATA_DIR / class_name
                    target_dir.mkdir(exist_ok=True)

                    # Dateien verschieben
                    files = list(class_dir.glob("*"))
                    for f in files:
                        shutil.move(str(f), str(target_dir / f.name))

            # Leeren Split-Ordner löschen (z.B. data/train)
            shutil.rmtree(split_path)
            print(f"  -> Ordner '{split}' aufgelöst und Dateien zurückgeschoben.")

    print("\n✅ MERGE ABGESCHLOSSEN. Alle Bilder liegen wieder flach in data/Klasse.")

In [2]:
# Führe dies aus, um die Ordner zu erstellen (Train/Val/Test)
split_data()

Gefundene Klassen zum Splitten: ['cup', 'pen', 'bottle', 'keyboard']
Split-Verhältnis: 70.0% / 15.0% / 15.0%

  -> cup: 70 Train, 15 Val, 15 Test
  -> pen: 70 Train, 15 Val, 15 Test
  -> bottle: 70 Train, 15 Val, 15 Test
  -> keyboard: 70 Train, 15 Val, 16 Test

✅ SPLIT ABGESCHLOSSEN. Ordnerstruktur ist jetzt bereit für Training.


In [ ]:
# Führe dies aus, um alles rückgängig zu machen (alles zurück in data/Klasse)
merge_data()